我们来训练一个简单的图像识别模型

一些前置准备：

In [5]:
import torchvision
import torch.nn
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [6]:
#初始化一个device
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(f"当前的device类型：{device}")

当前的device类型：cuda


In [7]:
# 准备数据集
train_data = torchvision.datasets.CIFAR10("datasets", train=True, transform=torchvision.transforms.ToTensor(),
                                          download=True)
test_data = torchvision.datasets.CIFAR10("datasets", train=False, transform=torchvision.transforms.ToTensor(),
                                         download=True)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
train_data_size = len(train_data)
test_data_size = len(test_data)
print(f"训练数据集的长度：{test_data_size}\n测试数据集的长度：{test_data_size}")

训练数据集的长度：10000
测试数据集的长度：10000


In [9]:
# 准备dataloader：
train_dataloader = DataLoader(train_data, 64)
test_dataloader = DataLoader(test_data, 64)

搭建一个神经网络

In [10]:
from torch import nn
from torch.nn import Sequential, Conv2d, MaxPool2d, Flatten, Linear


# 搭建神经网络：
class MyModule(nn.Module):
    """搭建神经网络，用于处理CIFAR10"""

    def __init__(self):
        super(MyModule, self).__init__()
        self.model1 = Sequential(
            Conv2d(3, 32, 5, 1, padding=2),
            MaxPool2d(2),
            Conv2d(32, 32, 5, 1, padding=2),
            MaxPool2d(2),
            Conv2d(32, 64, 5, 1, padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )
        self.type = ['飞机', '手机', '鸟', '猫', '鹿', '狗', '青蛙', '马', '船', '树干']

    def forward(self, x):
        x = self.model1(x)
        return x

In [11]:
mymodle = MyModule()
mymodle.to(device)

MyModule(
  (model1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=1024, out_features=64, bias=True)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [12]:
# 损失函数：
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn.to(device)
# 优化器：
learning_rate = 0.01
optim = torch.optim.SGD(mymodle.parameters(), lr=learning_rate)

In [13]:
# 声明一些计数变量
total_train_step = 0
total_test_step = 0
epoch = 2
# 添加tensorboard
writer = SummaryWriter("logs")

上面都是一些准备模块，接下来是对模型的训练：

In [15]:
for i in range(epoch):
    print(f"第{i + 1}轮训练开始：")
    # 训练步骤:
    for data in train_dataloader:
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        outputs = mymodle(imgs)
        # print(outputs)
        result_loss = loss_fn(outputs, targets)  # 损失函数的计算
        optim.zero_grad()  # 梯度归零
        result_loss.backward()  # 梯度反向传播
        optim.step()  # 优化器
        total_train_step += 1

        if total_train_step % 100 == 0:
            print(f"训练次数{total_train_step},当前loss:{result_loss.item()}")
            writer.add_scalar("train_loss", result_loss.item(), total_train_step)

    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        # 每次训练结束后用测试集验证一次结果，no_grad即不对梯度造成影响
        for data in test_dataloader:
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = mymodle(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss

            #正确率计算：
            accuracy = (outputs.argmax(1) == targets).sum()
            """outputs.argmax(1)将每一行（原结构是对每一类的预测概率），改为每一行最大值的index，实际代表预测的类别"""
            """== targets即将预测结果与目标结果对比，得到true、false组成的数组"""
            """.sum()即将结果求和，实际代表命中的次数"""
            total_accuracy += accuracy
            """将本loader中数据的命中次数加至总和"""
    print(f"第{i + 1}轮训练结束，整体测试集上的loss:{total_test_loss}")
    print(f"第{i + 1}轮训练结束，整体测试集上的正确率:{total_accuracy/test_data_size}")
    writer.add_scalar("test_loss", loss.item(), total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy/test_data_size, total_test_step)
    total_test_step += 1

writer.close()

torch.save(mymodle.state_dict(), f"mymodle-1.pth")
print("训练已完成，模型已保存！")

第1轮训练开始：
训练次数1600,当前loss:1.710988998413086
训练次数1700,当前loss:1.6389728784561157
训练次数1800,当前loss:1.9491033554077148
训练次数1900,当前loss:1.692307472229004
训练次数2000,当前loss:1.9116666316986084
训练次数2100,当前loss:1.5218448638916016
训练次数2200,当前loss:1.47725248336792
训练次数2300,当前loss:1.769529104232788
第1轮训练结束，整体测试集上的loss:266.0967712402344
第1轮训练结束，整体测试集上的正确率:0.3870999813079834
第2轮训练开始：
训练次数2400,当前loss:1.7246737480163574
训练次数2500,当前loss:1.3543579578399658
训练次数2600,当前loss:1.5739370584487915
训练次数2700,当前loss:1.6835168600082397
训练次数2800,当前loss:1.4994451999664307
训练次数2900,当前loss:1.5783699750900269
训练次数3000,当前loss:1.3887561559677124
训练次数3100,当前loss:1.5180506706237793
第2轮训练结束，整体测试集上的loss:264.96923828125
第2轮训练结束，整体测试集上的正确率:0.38760000467300415
训练已完成，模型已保存！


使用训练好的模型进行一下预测：

In [22]:
import torchvision.transforms as trans
from nn_CIFAR10 import MyModule
from PIL import Image

In [23]:
img_path = 'imgs/qingwa.png'
img = Image.open(img_path)
print(img.format)
#png是四通道，需要转通道
if img.format == 'PNG':
    img = img.convert("RGB")

PNG


In [24]:
transform2 = trans.Compose([trans.Resize((32, 32)),
                           trans.ToTensor()])
img = transform2(img)
#print(f"img.shape={img.shape}")
print(f"图片{img_path}读取并转换成功")

图片imgs/qingwa.png读取并转换成功


In [25]:
img

tensor([[[0.5529, 0.4941, 0.4902,  ..., 0.4745, 0.4824, 0.4902],
         [0.4980, 0.4902, 0.4980,  ..., 0.4745, 0.4863, 0.4863],
         [0.4902, 0.4824, 0.4784,  ..., 0.4824, 0.4824, 0.4941],
         ...,
         [0.5216, 0.5059, 0.4745,  ..., 0.5137, 0.5216, 0.5294],
         [0.5451, 0.5098, 0.4980,  ..., 0.5176, 0.5216, 0.5373],
         [0.5255, 0.4784, 0.5176,  ..., 0.5255, 0.5373, 0.5529]],

        [[0.8392, 0.8235, 0.8157,  ..., 0.7961, 0.8000, 0.8000],
         [0.8157, 0.8196, 0.8157,  ..., 0.7922, 0.8000, 0.8000],
         [0.8196, 0.8157, 0.8118,  ..., 0.8000, 0.8000, 0.8118],
         ...,
         [0.8196, 0.8039, 0.7608,  ..., 0.8471, 0.8549, 0.8627],
         [0.8549, 0.8353, 0.8039,  ..., 0.8471, 0.8549, 0.8667],
         [0.8510, 0.8196, 0.8118,  ..., 0.8588, 0.8627, 0.8745]],

        [[0.6392, 0.5569, 0.5569,  ..., 0.5647, 0.6039, 0.6431],
         [0.5882, 0.5647, 0.5765,  ..., 0.6000, 0.6196, 0.6314],
         [0.5647, 0.5529, 0.5529,  ..., 0.6078, 0.6157, 0.

In [26]:
module = MyModule()
module.load_state_dict(torch.load("mymodle.pth"))
print("模型加载成功")

模型加载成功


把处理好的图片，输入到之前训练的模型中，得到识别结果

In [27]:
img = torch.reshape(img, (1, 3, 32, 32))
module.eval()   #转换至测试状态
with torch.no_grad():
    outputs = module(img)
#转换一下测试结果
print(f"预测的结果是：{module.type[outputs.argmax(1)]}")

预测的结果是：青蛙
